---
{}

---





# core

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import cudaq
import numpy as np
from typing import List

# %% [markdown]
# ## String and Dictionary Utilities
# 
# These functions help with parsing CUDA-Q results into Qiskit-style count dictionaries.

# %%
#| export
def reverse_key(key: str) -> str:
    "Reverse a string key (used for bitstring manipulation)"
    return key[::-1]

#| export
def process_dict(d: dict) -> dict:
    """
    Reverse all keys in the dictionary and map them to corresponding values.
    """
    mapped_dict = {}
    reversed_dict = {reverse_key(k): v for k, v in d.items()}
    mapped_reversed_dict = {k: mapped_dict[k] for k in reversed_dict if k in mapped_dict}
    return mapped_reversed_dict

#| export
def string_to_dict(raw_string: str) -> dict:
    """
    Convert a raw CUDA-Q result string to a dictionary with reversed keys.
    """
    raw_string = ''.join(filter(lambda x: x.isdigit() or x == ':' or x.isspace(), raw_string))
    raw_list = raw_string.split()
    mapped_dict = {}
    for item in raw_list:
        key, value = item.split(":")
        mapped_dict[key] = int(value)
    rev = {reverse_key(k): v for k, v in mapped_dict.items()}
    return rev

#| export
def counts_cudaq_to_qiskit(resL: list) -> list:
    """
    Convert a list of CUDA-Q results into Qiskit-style counts dictionaries.
    """
    probsBL = [0] * len(resL)
    for i, res in enumerate(resL):
        buf = res.__str__()
        probsBL[i] = string_to_dict(buf)
    return probsBL

# %% [markdown]
# ## Qiskit → CUDA-Q Conversion
# 
# This function translates a Qiskit circuit into a CUDA-Q kernel.

# %%
#| export


# %% [markdown]
# ## CUDA-Q Execution Utilities
# 
# These functions run CUDA-Q kernels either sequentially or in parallel across multiple QPUs.

# %%
#| export
def cudaq_run_parallel_qpu(qKerL, shots: int, qpu_count: int):
    """
    Run multiple CUDA-Q kernels in parallel across available QPUs.
    """
    count_futures = {kernel: [] for kernel in qKerL} 
    for i, kernel in enumerate(qKerL):
        gpu_id = i % qpu_count
        count_futures[kernel].append(cudaq.sample_async(kernel, shots_count=shots, qpu_id=gpu_id))
    return [counts.get() for futures in count_futures.values() for counts in futures]

#| export
def cudaq_run(qKerL, shots: int):
    """
    Run multiple CUDA-Q kernels sequentially.
    """
    return [cudaq.sample(kernel, shots_count=shots) for kernel in qKerL]

# %% [markdown]
# ## CUDA-Q Kernel: `circ_kernel`
# 
# This kernel constructs a quantum circuit from a list of gates and parameters.

# %%
#| export
@cudaq.kernel
def circ_kernel(num_qubit: int, num_gate: int, gate_type: list[int], angles: list[float]):
    qvector = cudaq.qvector(num_qubit)
    for i in range(num_gate):
        j = 3 * i
        gateId = gate_type[j]
        q0 = qvector[gate_type[j+1]]
        
        if gateId == 1: h(q0)
        elif gateId == 2: ry(angles[j], q0)
        elif gateId == 3: rz(angles[j], q0)
        elif gateId == 4:
            q1 = qvector[gate_type[j + 2]]
            x.ctrl(q0, q1)
        elif gateId == 5:
            continue
        elif gateId == 6:
            q1 = qvector[gate_type[j + 2]]
            r1.ctrl(angles[j], q0, q1)
        elif gateId == 7:
            q1 = qvector[gate_type[j + 2]]
            swap(q0, q1)
        elif gateId == 8:
            theta, phi, lambda_ = angles[j], angles[j+1], angles[j+2]
            u3(theta, phi, lambda_, q0)

# %% [markdown]
# ## Quantum Fourier Transform (QFT) Kernels

# %%
#| export
@cudaq.kernel
def qft(qubits: cudaq.qview):
    qubit_count = len(qubits)
    for i in range(qubit_count):
        h(qubits[i])
        for j in range(i + 1, qubit_count):
            angle = (2 * np.pi) / (2**(j - i + 1))
            cr1(angle, [qubits[j]], qubits[i])

#| export
@cudaq.kernel
def inverse_qft(qubits: cudaq.qview):
    cudaq.adjoint(qft, qubits)

#| export
@cudaq.kernel
def qft_kernel(input_state: List[int]):
    qubit_count = len(input_state)
    qubits = cudaq.qvector(qubit_count)
    for i in range(qubit_count):
        if input_state[i] == 1:
            x(qubits[i])
    qft(qubits)

# %% [markdown]
# ## Qiskit → Gate List Conversion
# 
# This function converts a list of Qiskit circuits into a compact gate list representation for CUDA-Q.

# %%
#| export
def qiskit_to_gateList(qcL):
    nCirc = len(qcL)
    qc = qcL[0]
    nGate = len(qc)
    print('qiskit_to_gateList: nGate', nGate)

    circ_type = np.zeros(shape=(nCirc, 2), dtype=np.int32)
    gate_type = np.zeros(shape=(nCirc, nGate, 3), dtype=np.int32)
    gate_param = np.zeros(shape=(nCirc, nGate, 3), dtype=np.float32)

    m = {'h': 1, 'ry': 2, 'rz': 3, 'cx': 4, 'measure': 5, 'cp': 6, 'swap': 7, 'u': 8}

    for j, qc in enumerate(qcL):
        qregs = qc.qregs[0]
        nq = qc.num_qubits
        assert nGate >= len(qc)
        qregAddrD = {hex(id(obj)): idx for idx, obj in enumerate(qregs)}
        k = 0
        for op in qc:
            gate = op.operation.name
            params = op.operation.params
            qIdxL = [qc.find_bit(q).index for q in op.qubits]

            if gate == 'h':
                gate_type[j, k] = [m[gate], qIdxL[0], 0]
            elif gate in ['ry', 'rz']:
                gate_param[j, k, 0] = params[0]
                gate_type[j, k] = [m[gate], qIdxL[0], 0]
            elif gate in ['cx', 'cp', 'swap']:
                if gate == 'cp':
                    gate_param[j, k, 0] = params[0]
                gate_type[j, k] = [m[gate]] + qIdxL
            elif gate == 'u':
                gate_param[j, k] = params
                gate_type[j, k] = [m[gate], qIdxL[0], 0]
            elif gate in ['barrier', 'measure']:
                continue
            else:
                print('ABORT; unknown qiskit gate', gate)
                exit(99)
            k += 1
        circ_type[j] = [nq, k]
        
    outD = {'circ_type': circ_type, 'gate_type': gate_type, 'gate_param': gate_param}
    md = {'gate_map': m, 'num_qubit': nq, 'num_gate': nGate, 'num_circ': nCirc}
    return outD, md

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()